In [ ]:
import numpy
%matplotlib inline
from matplotlib import pyplot

In [ ]:
import sys
sys.path.append('../scripts/')

# Our helper, with the functions: 
# plot_vector, plot_linear_transformation, plot_linear_transformations
from plot_helper import *

## Geometry of eigendecomposition

When you know the eigenvectors and eigenvalues, you know the matrix. Many applications of linear algebra benefit from expressing a matrix using its eigenvectors and eigenvalues (it often makes computations easier). 
This is called _eigendecomposition_—matrix decomposition is a central topic in linear algebra, and particularly important in applications.

### Eigenvectors revisited

In the previous lesson, we saw that a unit circle, by a 2D linear transformation, lands on an ellipse. The semi-major and semi-minor axes of the ellipse are in the direction of the eigenvectors of the transformation matrix. Let's revisit that.


We'll work with the matrix $\,A = \begin{bmatrix} 2 & 1 \\ 1 & 2 \end{bmatrix}$.

In [ ]:
A = numpy.array([[2,1], [1,2]])
print(A)
plot_linear_transformation(A)

As in the previous lesson, plot a set of vectors of unit length (whose heads trace the unit circle), visualize the transformed vectors, then compute the length of the semi-major and semi-minor axes of the ellipse (the norm of the longest and shortest vectors in our set).

In [ ]:
alpha = numpy.linspace(0, 2*numpy.pi, 41)
vectors = list(zip(numpy.cos(alpha), numpy.sin(alpha)))
newvectors = []
for i in range(len(vectors)):
    newvectors.append(A.dot(numpy.array(vectors[i])))

plot_vector(vectors)

In [ ]:
plot_vector(newvectors)

In [ ]:
lengths = []
for i in range(len(newvectors)):
    lengths.append(numpy.linalg.norm(newvectors[i]))
semi_major = max(lengths)
print('Semi-major axis {:.1f}'.format(semi_major))
semi_minor = min(lengths)
print('Semi-minor axis {:.1f}'.format(semi_minor))

u1 = numpy.array([semi_major/numpy.sqrt(2), semi_major/numpy.sqrt(2)])
u2 = numpy.array([-semi_minor/numpy.sqrt(2), semi_minor/numpy.sqrt(2)])

### A scaling transformation

OK, cool. In our first lesson, we saw some special transformations: _rotation_, _shear_, and _scaling_. 
Looking at the effect of the matrix transformation $A$ on the unit circle, we might imagine obtaining the same effect by first scaling the unit vectors—stretching $\mathbf{i}$ to $3\times$ its length and leaving $\mathbf{j}$ with length $1$—and then rotating by 45 degrees counter-clockwise.
We have also learned that applying linear transformations in sequence like this amounts to matrix multiplication.

Let's try it. We first define the scaling transformation $S$, and apply it to the vectors mapping the unit circle. 

In [ ]:
S = numpy.array([[3,0], [0,1]])
print(S)

In [ ]:
ellipse = []
for i in range(len(vectors)):
    ellipse.append(S.dot(numpy.array(vectors[i])))

plot_vector(ellipse)

### A rotation transformation

We figured out the matrix for a 90-degree rotation in our first lesson. But how do you rotate by any angle? You never have to memorize the "formula" for a rotation matrix. Just think about where the unit vectors land. Look at the figure below, and follow along on a piece of paper if you need to.

<img src="../images/rotation.png" style="width: 300px;"/> 
#### Rotation of unit vectors by an angle $\theta$ to the left.

$$
\mathbf{i} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}  \Rightarrow  \begin{bmatrix} \cos{\theta} \\ \sin{\theta} \end{bmatrix} \\
\mathbf{j} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}  \Rightarrow  \begin{bmatrix} -\sin{\theta} \\ \cos{\theta} \end{bmatrix}
$$

You now can build the rotation matrix using the column vectors where each unit vector lands.

$$R = \begin{bmatrix} \cos{\theta} & -\sin{\theta} \\ \sin{\theta} & \cos{\theta} \end{bmatrix}$$

Great. Let's define a matrix $R$ that rotates vectors by 45 degrees.

In [ ]:
theta = numpy.pi/4
R = numpy.array([[numpy.cos(theta), -numpy.sin(theta)], 
                 [numpy.sin(theta), numpy.cos(theta)]])

We can apply this rotation now to the `ellipse` vectors, and plot the result.

In [ ]:
rotated = []
for i in range(len(vectors)):
    rotated.append(R.dot(numpy.array(ellipse[i])))

plot_vector(rotated)

### Composition of the transformations

It certainluy _looks_ like we recovered the picture we obtained originally when applying the transformation $A$ to all our vectors on the unit circle.  

But have a look at the two transformations—the scaling $S$ and the rotation $R$—applied in sequence:

In [ ]:
plot_linear_transformations(S,R)

Observe carefully the plot above. The scaling did stretch the basis vector $\mathbf{i}$ by $3\times$ its original length. It also left the basis vector $\mathbf{j}$ with its length equal to $1$. But something looks _really_ off after the second transformation. 

We know from the discussion in the previous lesson that the vector that lands on the ellipse's semi-major axis doesn't change direction. It's _not_ the basis vector $\mathbf{i}$ that lands there, it's the vector $\mathbf{v}_1$ that satisfies: 

$$ A \mathbf{v}_1 = s_1 \mathbf{v}_1 $$

Recalling the process we followed in the previous lesson, we find that vector, and plot it together with its transformed version (also called its _image_):

In [ ]:
A_inv = numpy.linalg.inv(A)
v1 = A_inv.dot(u1)
plot_vector([u1,v1])

Right. The unit vector that was aligned with the 45-degree line got transformed onto the semi-major axis of the ellipse, without being rotated. This is the effect of the matrix $A$ on $\mathbf{v}_1$: _it is just scaled_.

Now, let's look at the sequence of transformations $S$ and $R$ applied to $\mathbf{v}_1$. We apply the transformations by matrix-vector multiplication, and in the second step, we use composition of transformations.

In [ ]:
plot_vector([v1, S.dot(v1)])

In [ ]:
plot_vector([S.dot(v1),R.dot(S.dot(v1))])

That is definitely _not_ what we expected. Oh well. It seemed like a good idea at the time, but the scaling $S$ and the rotation $R$ applied in sequence are _not_ equivalent to the transformation $A$. 
Our visual intuition was not able to get the whole picture.

### Complete the composition

OK. This will blow your mind… to get the same transformation as $A$ we had to _first_ rotate 45 degrees to the right (which leaves the plot of our circle unchanged even though the vectors rotated), _then_ scale, and finally rotate 45 degrees to the left. 

We will look at this sequence of transformations via matrix multiplicaton. But first note that a rotation by a negative angle $\theta$ is achieved by the matrix:

$$R^T = \begin{bmatrix} \cos{\theta} & \sin{\theta} \\ -\sin{\theta} & \cos{\theta} \end{bmatrix}$$

Check using a piece of paper that the columns of this matrix make sense for a negative angle $\theta$, and notice that it is the transpose of $R$ (i.e., swaps rows and columns).

Now look at the result of the matrix multiplication (equivalent to the transformations in sequence, right to left):

In [ ]:
R @ S @ R.transpose()

That's certainly the same as $A$!

In [ ]:
print(A)

In [ ]:
plot_linear_transformation(R @ S @ R.transpose())

We have some explaining to do. Let's visualize the transformation $R^T$, adding to our plot the unit vectors that were aligned with the eigenvectors, $\mathbf{v}_1$ and $\mathbf{v}_2$. You see that they land on the coordinate axes.

In [ ]:
v2 = A_inv.dot(u2)
plot_linear_transformation(R.transpose(), v1, v2)

Now let's visualize applying  the scaling transformation to these vectors, and applying the rotation matrix $R$ after that.

In [ ]:
e1 = R.transpose().dot(v1)
e2 = R.transpose().dot(v2)

plot_linear_transformation(S, e1, e2)

In [ ]:
plot_linear_transformation(R, S.dot(e1), S.dot(e2))

Satisfied? The vectors $\mathbf{v}_1$ and $\mathbf{v}_2$ are first rotated to land on the axes, are then scaled, and are finally rotated back to their original direction. This has the same effect as the transformation $A$. In other words:


$$ A = R\, S\, R^T 
$$

The equivalence above shows an **eigendecomposition** of the matrix $A$. 
The scaling matrix $S$ has the *eigenvalues* along the diagonal, and the rotation matrix $R$ has the *eigenvectors* as columns. Let's check:

In [ ]:
print(R[:,0])
print(v1)

In [ ]:
print(R[:,1])
print(v2)

### Symmetric matrices, orthogonal eigenvectors

In our example, we figured out that the matrix that undoes the effect of the positive rotation $R$ is its transpose, $R^T$. But in general, the matrix that has the effect of undoing a transformation is the _inverse_ of this matrix. In fact, $R$ is special, because

$$ R^T = R^{-1} $$

##### Definition:

> A matrix $R$ whose transpose is equal to its inverse is called **orthogonal**.

The columns of an orthogonal matrix are orthogonal vectors of unit length (a.k.a., _orthonormal_). For our example, it means that the _eigenvectors of $A$ are orthogonal_.
This always happens with _symmetric matrices_. 

In [ ]:
print(R.transpose())

In [ ]:
print(numpy.linalg.inv(R))

You can check visually that $R^T$ is equal to $R^{-1}$—but if we try a logical expression, it will give `False` because of floating-point errors. To check for the symmetry of $A$, we can use a logical operation:

In [ ]:
A == A.transpose()

And to confirm that the two eigenvectors are orthogonal, we take the dot product, which is zero for perpendicular vectors:

In [ ]:
v1.dot(v2)

##### Key idea:

> A _symmetric_ matrix, $\,A = A^T$, has _orthogonal_ eigenvectors, and can be decomposed as $A = R\, S\, R^T$, where $R$ has the eigenvectors as columns and $S$ is the diagonal matrix of eigenvalues.

## Eigendecomposition in general

Let's work the general case. For the two eigenvectors of $A$:

$$
\begin{align*}
  A \mathbf{v_1} = s_1 \mathbf{v_1} \\
  A \mathbf{v_2} = s_2 \mathbf{v_2}
\end{align*}
$$

The two left-hand sides are matrix-vector multiplications, giving a vector as result. They can be arranged as two column vectors. 
The two right-hand sides are a scalar multiplying a column vector.
Stack the two equations together like this: 

$$
  A \begin{bmatrix}
    \mathbf{v_1} & \mathbf{v_2}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \mathbf{v_1} & \mathbf{v_2}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 \\
    0 & s_2
    \end{bmatrix}  
$$

Using $C$ to denote the matrix of eigenvectors, and $D$ to denote the diagonal matrix of eigenvalues, it becomes:

$$
  A\, C = C\, D
$$

then right-multiply by $C^{-1}$ on both sides:

$$
  A = C\, D\, C^{-1}
$$

A matrix that can be decomposed in this way is called **diagonalizable**.
Multiplying on the right by $C$ and on the left by $C^{-1}$:

$$
  C^{-1} A\, C = D
$$


If you go back to the previous lesson, you will find an expression that looks like this for applying a known transformation to a vector in a new basis. Review that section if you need to.

Applying the transformation $A$ to a vector in standard basis, $\mathbf{x}$, is:

$$
  A\, \mathbf{x} = C\, D\, C^{-1}\mathbf{x}
$$

Viewing $C$ as a change of basis, the expression on the right means we change $\mathbf{x}$ to a new basis of eigenvectors, apply a scaling by the eigenvalues in the new coordinate system, and change back to the standard basis. The effect is the same as the transformation $A$: the matrices $A$ and $D$ are called **similar**.

##### Key idea:

> Similar matrices have the same effect, via a change of basis.

## Compute eigenthings in Python

You can compute the eigenvalues and eigenvectors of a matrix using [`numpy.linalg.eig()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html). It returns a tuple: its first element is an array with the eigenvalues, and its second element is a 2D array  where each column is an eigenvector.

In [ ]:
numpy.linalg.eig(A)[0]

In [ ]:
numpy.linalg.eig(A)[1]

In [ ]:
# display each eigenvalue with the corresponding eigenvector, side by side
eigenvalues, eigenvectors = numpy.linalg.eig(A)

for eigenvalue, eigenvector in zip(eigenvalues, eigenvectors.T):
    print(eigenvalue, eigenvector)

To create the diagonal matrix of eigenvalues, use [`numpy.diag()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.diag.html): if you give it a 1D array, it returns a 2D array with the elements of the input array in the diagonal. 

The eigendecomposition is shown below: 

In [ ]:
C = eigenvectors
A_decomp = C @ numpy.diag(eigenvalues) @ numpy.linalg.inv(C)
print(A_decomp)

Another way to compute all the eigenthings is to use **SymPy**: the Python library for symbolic computations (a.k.a., computer algebra system). SymPy has a [`Matrix`](https://docs.sympy.org/latest/tutorial/matrices.html) data type with many advanced methods. 

You will create a `Matrix` from a list of row vectors, and then use the `diagonalize()` method to obtain the matrix of eigenvectors, and the diagonal matrix of eigenvalues. 
SymPy can display beautiful symbolic mathematics. Check it out.

In [ ]:
import sympy

sympy.init_printing(use_latex = 'mathjax') #configures the display of mathematics

In [ ]:
A = sympy.Matrix([[2,1], [1,2]])
A

In [ ]:
A == A.transpose()

In [ ]:
C, D = A.diagonalize()
D

With SymPy, you can also get just the eigenvalues using the [`eigenvals()`](https://docs.sympy.org/latest/tutorial/matrices.html#eigenvalues-eigenvectors-and-diagonalization) method. Its output will be in the form $\{s_1:m_1, s_2:m_2\}$, where $s_1$, $s_2$ are the eigenvalues of the matrix, and $m_1$, $m_2$ are the _multiplicities_. A matrix can have (multiply) repeated eigenvalues sometimes. In the case of our matrix $A$, both eigenvalues are unique (i.e., have multiplicity $1$).

In [ ]:
A.eigenvals()

But think of the identity matrix: it has $1$ along the diagonal, and the two eigenvalues are the same, $1$. We can create an identity matrix in SymPy using `eye(n)`, with `n` indicating the dimension.

In [ ]:
sympy.eye(2).eigenvals()

If we try with the _shear_ matrix from our fist lesson, we see that it also has a repeated $1$ eigenvalue.

In [ ]:
shear = sympy.Matrix([[1,1], [0,1]])
shear

In [ ]:
shear.eigenvals()

Let's have a look back at the equation for the eigenvectors—on which the effect of a matrix $A$ is just to scale them—and rearrange things a bit:

$$ \begin{align*}
  A\, \mathbf{v} &= s\, \mathbf{v}\\
  A\, \mathbf{v} - s\, \mathbf{v} &= \mathbf{0} \\
  (A - s\, I) \mathbf{v} &= \mathbf{0}
\end{align*}
$$

The last form represents a homogeneous linear system, whose solutions are the _nullspace_ of $(A-s\,I)$, as we saw in our second lesson. We can compute the nullspace using SymPy; let's try it with the shear matrix:

In [ ]:
(shear - sympy.eye(2)).nullspace()

The [`nullspace()`](https://docs.sympy.org/latest/tutorial/matrices.html#nullspace) method of SymPy matrices returns a list of column vectors that span the nullspace of the matrix. In the case of the shear matrix, only one column vector is returned, which means  its nullspace is a line.

The shear matrix has one repeated eigenvalue, and a single eigenvector: we cannot build a change of basis with its eigenvectors, which means it's _not diagonalizable_.

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())